# Tutorial on SimCSE

- 作成者: Shunsuke Kanda ([@kampersanda](https://github.com/kampersanda))
- 作成日: 2023-10-29

## SimCSEについて

SimCSEは、対照学習を用いた文埋め込み技術です。

> Tianyu Gao, Xingcheng Yao, and Danqi Chen. [SimCSE: Simple Contrastive Learning of Sentence Embeddings](https://aclanthology.org/2021.emnlp-main.552/). EMNLP 2021.

SimCSEは、簡単なアルゴリズムでラベルの無い文集合から文埋め込みモデルを学習することができます。
その文埋め込みは、Semantic Textual Similarity (STS) 評価タスクにおいて、教師ありのSentence-BERTと同程度の性能を示します。

また、正例ペアから成る訓練セットを用いて教師あり学習することで、更にその性能を向上することができます。
訓練セットの作り方次第では、それぞれの目的に応じた文埋め込みモデルを獲得することも可能です。

SimCSEは、実装の容易さ・応用の容易さ・高い性能などから、研究でも実用でも非常に有用な文埋め込み技術のひとつです。その実装方法を習得することは、自然言語処理や情報検索エンジニアにとって有益だと考えます。

## この資料について

### 動機

SimCSEのアイデアはシンプルなので、そのアルゴリズムを理解することはあまり難しくないです。良い教材もネットに揃っています。

しかし、SimCSEを実装し応用できるようになるには、深層学習フレームワークや自然言語処理について一定の知識と経験が必要になります。例えば、ある深層学習アルゴリズムの実装を眺めてみて、ライブラリの使用方法や、当たり前に記述されているヒューリスティックの意味が分からず、一行一行調べながらコードを読んだ経験のある方も多いと思います。

また、初学者の方にとってはGPUなどの環境構築も1つのハードルであり、Google Colaboratoryなどの環境で試せることも重要でしょう。

### 目的

この資料は、SimCSEについて上記のような問題を解決することを目的とし、SimCSEの学習から評価まで一連の実装とその解説をNotebookで提供します。

### 特徴

- 以下の一連の処理を、上から順に実行することで簡単に試すことが可能です
    - データセットの準備
    - モデルの定義
    - モデルの学習
    - モデルの評価
- PyTorchやTransformersの基本的な使用方法や、自然言語処理でよく知られるヒューリスティックも解説します
- 教師なしと教師ありの両方の実装を提供します

### 想定する利用者

- SimCSEのアイデアは理解できるが、深層学習フレームワークなどの経験の少なさから実際にモデルを実装するのにはハードルを感じるという方
- 業務などでSimCSEのアルゴリズムを実装する必要がある方

### 読むのに必要なこと

SimCSEの目的と基本的なアイデアを理解していることを前提とします。これらの知識習得のために、以下のスライド資料をオススメします。

> [[輪講資料] SimCSE: Simple Contrastive Learning of Sentence Embeddings](https://speakerdeck.com/hpprc/lun-jiang-zi-liao-simcse-simple-contrastive-learning-of-sentence-embeddings-823255cd-bd1f-40ec-a65c-0eced7a9191d)

また、深層学習モデルの基本的な学習方法（ミニバッチ学習など）も既知を前提とします。

### 作成方法

SimCSEのシンプルな再実装 [hppRC/simple-simcse](https://github.com/hppRC/simple-simcse) が存在します。こちらのレポジトリは、SimCSEの学習と評価アルゴリズムの簡潔な実装を提供しており、コードの各パートにも丁寧な解説コメントを記述しています。深層学習の経験があり、論文を読んでその内容が理解できる方にとってはsimple-simcseが必要十分な資料だと思います。

このNotebookでは更に基礎的な部分からの解説と簡単な利用を試み、作成者が解説コメントを追記しつつ、simple-simcseの内容をNotebookで再実装しました。また、教師あり学習のパートも追加しました。

## 参考資料

以下を引用しつつ解説します。

- Tianyu Gao, Xingcheng Yao, and Danqi Chen. [SimCSE: Simple Contrastive Learning of Sentence Embeddings](https://aclanthology.org/2021.emnlp-main.552/). EMNLP 2021. ("論文"として引用)
- https://github.com/princeton-nlp/SimCSE. ("オリジナルの実装"として引用)
- [[輪講資料] SimCSE: Simple Contrastive Learning of Sentence Embeddings](https://speakerdeck.com/hpprc/lun-jiang-zi-liao-simcse-simple-contrastive-learning-of-sentence-embeddings-823255cd-bd1f-40ec-a65c-0eced7a9191d) by Hayato Tsukagoshi ("スライド"として引用)
- 岡﨑, 荒瀬, 鈴木, 鶴岡, and 宮尾. [IT Text 自然言語処理の基礎](https://www.ohmsha.co.jp/book/9784274229008/), 2022. ("岡﨑ら本"として引用)

## Notebookの構成

Notebookは以下の4章で構成されます。

1. 共通の設定
2. 教師なし学習（unsup-SimCSE）
3. 教師あり学習（sup-SimCSE）
4. 評価

上から順に実行する想定ですが、2と3は任意で実行をスキップしても機能します。Google Colaboratoryで実行されることを想定します。

## クレジット

このNotebookは、LegalOn Technologiesの社内勉強会で使用した資料です。検索チームが主催するセマンティック検索とベクトル検索に関する勉強会の発表資料として作成されました。

このNotebookの実装とコメントの大部分は、[hppRC/simple-simcse](https://github.com/hppRC/simple-simcse)からの移植です。

このNotebookは、[Apache License Version 2.0](https://www.apache.org/licenses/LICENSE-2.0)に準拠します。

## 謝辞

このNotebookは、[hppRC/simple-simcse](https://github.com/hppRC/simple-simcse)と上記スライド無しでは作成できませんでした。これらの制作者である塚越駿さんに感謝致します。

同僚の小林さんと藤田さんにも資料作成にあたって有益なコメントを頂きました。感謝致します。

# 1. 共通の設定

In [1]:
!pip install transformers==4.34.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.9 MB/s eta 0:00:00


In [2]:
import csv
import os
from typing import Callable

import more_itertools
import pandas as pd
import scipy
from sklearn import metrics as sklearn_metrics
import torch
import tqdm
import transformers

In [3]:
# モデルを保存するためにGoogle Driveをマウントしておく
from google.colab import drive
drive.mount('/content/drive')

# モデルの保存先パス (ファイル名の衝突に注意)
unsup_model_path = './drive/MyDrive/unsup-simcse-model.pth'
sup_model_path = './drive/MyDrive/sup-simcse-model.pth'

Mounted at /content/drive


In [4]:
# SimCSEモデルクラスの定義

# torch.nn.Moduleのサブクラスとして、SimCSEモデルを定義
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html
class SimCSEModel(torch.nn.Module):

    # SimCSEに使用する事前学習済みモデル名をリストで管理
    # 論文と同じくBERTとRoBERTaモデルを想定
    SUPPORTED_MODELS = ['bert-base-uncased', 'bert-large-uncased', 'roberta-base', 'roberta-large']

    # 内部で使用するTransformersのモデル名を受け取る
    def __init__(self, model_name: str) -> None:
        if not model_name in self.SUPPORTED_MODELS:
            raise ValueError(f'{model_name} is not supported.')

        # 親クラスの__init__()を最初に呼び出す仕様
        super().__init__()

        # SimCSEに使用する事前学習済みTransformersモデルをインスタンス化
        #
        # Automodelでモデル名からLookupしてモデルをダウンロードし読み込んでくれる
        # https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModel
        self.backbone: transformers.modeling_utils.PreTrainedModel = transformers.AutoModel.from_pretrained(model_name)

        # 追加で多層パーセプトロン(MLP)層を定義
        #
        # backboneから得られる埋め込みを更に変換する
        # 性能改善のためのオプショナルなコンポネントなので、無くても機能する
        #
        # 論文6.3節を参照
        # オリジナルの実装は以下を参照
        # https://github.com/princeton-nlp/SimCSE/blob/0.4/simcse/models.py#L19
        self.hidden_size: int = self.backbone.config.hidden_size
        self.dense = torch.nn.Linear(self.hidden_size, self.hidden_size)
        self.activation = torch.nn.Tanh()

    # 入力文のトークン列を受け取り、その文埋め込みを返す
    #
    # 引数はそのままBERT/RoBERTaモデルに引き渡される
    # 引数の意味はBertModelのforwardを参照
    # https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor = None,
        # RoBERTa variants don't have token_type_ids, so this argument is optional
        token_type_ids: torch.Tensor = None,
    ) -> torch.Tensor:
        # input_ids.size() = (batch_size, seq_len)
        # attention_mask.size() = (batch_size, seq_len)
        # token_type_ids.size() = (batch_size, seq_len)

        # BERT/RoBERTaモデルで推論
        outputs = self.backbone.forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        # 推論結果から文埋め込みを抽出
        #
        # BERTモデルをファインチューニングする場合は、Transformerの[CLS]トークンに対応した最終層の隠れ状態ベクトルを
        # 文の埋め込み表現として用いることが多い (岡崎ら本7.3節参照)
        # [CLS]トークン: 入力文の先頭に付随する特殊トークンで、文全体を表現する役割として分類問題などで使用される
        #
        # outputs.last_hidden_state.size() = (batch_size, seq_len, hidden_size)
        # emb.size() = (batch_size, hidden_size)
        emb = outputs.last_hidden_state[:, 0]

        # 上の代わりに、全サブワードに対応する最終層の隠れ状態ベクトルの平均プーリングや最大プーリングなどを用いても良い
        # オリジナルの実装では4種類を試している
        # https://github.com/princeton-nlp/SimCSE/blob/0.4/simcse/models.py#L63

        # unsup-SimCSEの場合、訓練時のみMLP層を使用するのが最も性能が良いという報告
        # sup-SimCSEの場合は、推論時でもMLP層を使用するか、もしくはMLP層自体を使用しない方が良い性能
        # 論文6.3節を参照
        #
        # (コメント) unsup-SimCSEでは学習データに適合し過ぎないで欲しいお気持ちがある？
        #
        # self.trainingのON/OFFはtorch.nn.Module.train()/.eval()で制御可能
        if self.training:
            emb = self.dense(emb)
            emb = self.activation(emb)

        # emb.size() = (batch_size, hidden_size)
        return emb

In [5]:
# 使用する計算デバイスの設定
# cpuでは低速すぎるので、基本的にはcudaを使用する

# device = 'cpu'
device = 'cuda'

# 2. 教師なし学習（unsup-SimCSE）

## 2.1 モデルインスタンスの生成

In [6]:
# Transformersの事前学習済みモデルからインスタンスを生成

# この例ではベーシックなBERTモデルのbert-base-uncasedを使ってみる
model_name = 'bert-base-uncased'

# SimCSEModelをインスタンス化し、指定したデバイスに載せる
model = SimCSEModel(model_name).to(device)

# テキストをトークンに分割しモデルに入力できる形式に変換するためのトークナイザを生成する
tokenizer: transformers.tokenization_utils.PreTrainedTokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

## 2.2 データセットの準備

In [7]:
# unsup-SimCSE訓練用のデータセットをダウンロード

# 論文で実際に使用されたEnglish Wikipediaデータセットが使用できる
# ランダムに抽出された100万行の英文章から構成される
!mkdir -p ./datasets/unsup-simcse
!wget https://huggingface.co/datasets/princeton-nlp/datasets-for-simcse/resolve/main/wiki1m_for_simcse.txt
!mv ./wiki1m_for_simcse.txt ./datasets/unsup-simcse/train.txt

--2023-11-01 09:02:22--  https://huggingface.co/datasets/princeton-nlp/datasets-for-simcse/resolve/main/wiki1m_for_simcse.txt
Resolving huggingface.co (huggingface.co)... 99.84.66.72, 99.84.66.112, 99.84.66.70, ...
Connecting to huggingface.co (huggingface.co)|99.84.66.72|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/princeton-nlp/datasets-for-simcse/7b1825863a99aa76479b0456f7c210539dfaeeb69598b41fb4de4f524dd5a706?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27wiki1m_for_simcse.txt%3B+filename%3D%22wiki1m_for_simcse.txt%22%3B&response-content-type=text%2Fplain&Expires=1699088543&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5OTA4ODU0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9wcmluY2V0b24tbmxwL2RhdGFzZXRzLWZvci1zaW1jc2UvN2IxODI1ODYzYTk5YWE3NjQ3OWIwNDU2ZjdjMjEwNTM5ZGZhZWViNjk1OThiNDFmYjRkZTRmNTI0ZGQ1YTcwNj9yZXNwb25zZS1jb25

In [8]:
# データセットは単純な行区切りのテキストデータ
with open('./datasets/unsup-simcse/train.txt') as f:
    sentences = [line.rstrip('\n') for line in f]

# 表示のためにDataFrameに変換
train_examples = pd.DataFrame(sentences, columns=['sentences'])
train_examples

,sentences
0,YMCA in South Australia
1,South Australia (SA) has a unique position in...
2,"The compound of philosophical radicalism, evan..."
3,It was into this social setting that in Februa...
4,"for apprentices and others, after their day's ..."
...,...
999995,Rubaschow: Roman.
999996,"Typoskript, März 1940, 326 pages."""
999997,"He deemed the discovery important because """"Da..."
999998,"In 2018, he reported that Elsinor Verlag (publ..."


In [9]:
# PyTorchのDatasetとDataLoaderを使ってデータセットを処理する
#
#  - torch.utils.data.Dataset: 訓練データを格納しアクセスするためのクラス
#  - torch.utils.data.DataLoader: 訓練データをイテレートするためのクラス（ミニバッチ化や再シャッフルなどを提供）
#
# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

# Datasetクラスを定義する
#
# __init__, __len__, __getitem__関数を実装すれば良い
# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files
class UnsupSimCSEDataset(torch.utils.data.Dataset):
    def __init__(self, sentences: list[str]) -> None:
        self.sentences = sentences

    # 事例の数を返す
    def __len__(self) -> int:
        return len(self.sentences)

    # idx番目の事例を返す
    def __getitem__(self, idx: int) -> str:
        return self.sentences[idx]


# 時間が掛かりすぎるので訓練事例を10万件に抑える
train_examples = train_examples[:100000]

# Datasetインスタンスを生成
train_dataset = UnsupSimCSEDataset(train_examples['sentences'].tolist())

In [10]:
# 上で作ったDatasetについてDataLoaderを作成

# ミニバッチのサイズ
#
# 論文で実際に使用されたモデルごとのバッチサイズの一覧は以下で提供されている
# https://github.com/princeton-nlp/SimCSE/tree/0.4#training
batch_size = 64

# DataLoaderでDatasetからフェッチされた部分データについて、ミニバッチを形成するための前処理を記述できる
# トークナイザを用いて、文をTransformersモデルに入力できる形式に変換する
#
# パラメータはオリジナルの実装に由来
# https://github.com/princeton-nlp/SimCSE/blob/0.4/run_unsup_example.sh
def collate_fn(batch: list[str]) -> transformers.tokenization_utils.BatchEncoding:
    return tokenizer(
        batch,
        # トークン列の長さをミニバッチ内の最大長に揃える
        padding='longest',
        # トークン列長がmax_lengthを超える場合は、末尾トークンを取り除きmax_lengthに揃える
        truncation='longest_first',
        # トークン列の最大長を指定
        max_length=32,
        # 結果をtorch.Tensor型で受け取る
        return_tensors='pt',
    )

# DataLoaderインスタンスを生成
# https://pytorch.org/docs/stable/data.html
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    collate_fn=collate_fn,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    # GPUへのメモリコピーを高速化する設定
    # https://stackoverflow.com/questions/55563376/pytorch-how-does-pin-memory-work-in-dataloader
    pin_memory=True,
    # 最後のバッチのサイズがbatch_sizeで割り切れない場合は、異なるサイズのバッチが生成されないように切り捨てる
    drop_last=True,
)

## 2.3 ファインチューニング

In [11]:
# 主な学習パラメータ

# エポック数 i.e., 訓練データを何回繰り返して学習するか
# 論文でのunsup-SimCSEのエポック数は1 (付録A参照)
epochs = 1

# 学習率: 各学習ステップにおけるパラメータ更新の幅で、小さいほど細かい調整となる
# 論文で実際に使用されたモデルごとの学習率の一覧は以下で提供されている
# https://github.com/princeton-nlp/SimCSE/tree/0.4#training
learning_rate = 3e-5

# 出力の確率分布の形状を制御するための温度パラメータ
# 論文の式(1)のτに相当する (解説は実際に使用されている箇所で後ほど)
temperature = 0.05

In [12]:
# 学習プロセス

# torch.optimを通してモデルのパラメータを更新する
#
# パラメータはtorch.nn.Module.parameters()で受け渡せば、後はoptimizerが管理を請け負ってくれる
# https://pytorch.org/docs/stable/optim.html
#
# TransformersのBERT/RoBERTaモデルはtorch.nn.Moduleのサブクラスなので、
# SimCSEModel.backboneのパラメータもparameters()で再帰的にイテレートされる
# https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
# https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaModel
#
# オリジナルの実装では、transformers.Trainerのデフォルト値であるAdamWを使用している
# https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.optimizers
#
# TransformersにもAdamWの実装があるが、現在は非推奨
# https://github.com/huggingface/transformers/issues/3407
# https://github.com/huggingface/transformers/issues/18757
optimizer = torch.optim.AdamW(
    params=model.parameters(),
    lr=learning_rate
)

# 学習ステップに応じて学習率を変動させるためのスケジューラを設定する
#
# オリジナルの実装では、transformers.Trainerのデフォルト値であるLinearSchedulerを使用している
# https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.optimizers
#
# これはステップ数に応じて線形に学習率がゼロに近づくスケジューラで、序盤は大胆に、終盤はきめ細かくパラメータを更新する
# https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.get_linear_schedule_with_warmup
#
# ただし、学習初期は予測がランダムで勾配も大きくなりやすいから、学習率は小さく抑えた方が良いというヒューリスティックも存在する
# そのためのアイデアが学習率のウォームアップで、序盤から中盤にかけて学習率を徐々に大きくしていく
# 参考：岡崎ら本6.4.2「学習率のウォームアップ」
#
# num_warmup_stepsでウォームアップのためのステップ数を指定できる
# ただし、オリジナルの実装ではtransformers.Trainerのデフォルト値を使用しているのでnum_warmup_steps=0
# https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.TrainingArguments
lr_scheduler = transformers.optimization.get_linear_schedule_with_warmup(
    optimizer=optimizer,
    # とりあえずここではオリジナルの実装に合わせてウォームアップ無し
    num_warmup_steps=0,
    # len(train_dataloader) is the number of steps in one epoch
    num_training_steps=len(train_dataloader) * epochs,
)

# 訓練モードに設定
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.train
model.train()

# ファインチューニング
for epoch in range(epochs):
    for batch in tqdm.tqdm(train_dataloader):
        # ミニバッチをデバイスに載せる
        batch = batch.to(device)

        # unsup-SimCSEのメインの学習処理
        #
        # 同じバッチの埋め込みを2回計算しているだけ
        # ただし内部では異なるドロップアウトが適用されているため、異なるデータ拡張による正例ペアが得られている
        # https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
        emb1 = model.forward(**batch)
        emb2 = model.forward(**batch)

        # (余談) 例えばtransformersのBERTモデルでは、実際にドロップアウト層が組み込まれていることが以下から確認できる。
        #
        # https://github.com/huggingface/transformers/blob/v4.34.0/src/transformers/models/bert/modeling_bert.py#L192
        # https://github.com/huggingface/transformers/blob/v4.34.0/src/transformers/models/bert/modeling_bert.py#L261
        #
        # ドロップアウトする確率はconfig.{hidden_dropout_prob, attention_probs_dropout_prob}で指定されており、以下の手順で変更できる
        # https://stackoverflow.com/questions/64947064/transformers-pretrained-model-with-dropout-setting

        # 全対コサイン類似度計算 such that
        #
        #  - sim_matrix.size() = (batch_size, batch_size)
        #  - sim_matrix[i][j] = cosine_sim(emb1[i], emb2[j])
        #
        # つまり、スライドP25の左の行列を作っている
        #
        # なぜこのコードで全対が計算できているかは、以下の解説が参考になる
        # https://medium.com/@dhruvbird/all-pairs-cosine-similarity-in-pytorch-867e722c8572
        sim_matrix = torch.nn.functional.cosine_similarity(emb1.unsqueeze(1), emb2.unsqueeze(0), dim=-1)

        # 温度パラメータによる確率分布の形状の調整
        #
        # sim_matrixは、この後のcross_entropyにてsoftmaxにより確率分布に変換される
        # その際に、温度パラメータが1より小さいと高い類似度を強調するように調整できる
        # https://qiita.com/nkriskeeic/items/db3b4b5e835e63a7f243
        #
        # SimCSEの性能は温度パラメータに敏感なので、適切な値を設定する必要がある
        # 論文では0.05が良かったと報告している (付録D参照)
        sim_matrix = sim_matrix / temperature

        # sim_matrixについて交差エントロピー損失を計算
        # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss
        #
        # SimCSEの目的関数は正例間のコサイン類似度を最大化すること
        #
        # ここでは、sim_matrixをクラス数batch_sizeな分類問題の推論結果と見なして、
        # 各行sim_matrix[i,:]が正解クラスに高い類似度、不正解クラスに低い類似度を予測できているかを評価している
        #
        # sim_matrixは対角成分に正例同士の類似度を格納しているので、行sim_matrix[i,:]の正解クラスとはsim_matrix[i,i]
        # labels=[0,1,2,...,batch_size-1]で各行の正解クラスを指定している
        labels = torch.arange(batch_size).long().to(device)
        loss = torch.nn.functional.cross_entropy(sim_matrix, labels)

        # 全ての勾配をゼロに初期化
        # https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
        optimizer.zero_grad()

        # 誤差逆伝播により勾配を計算
        loss.backward()

        # パラメータを更新
        optimizer.step()

        # 学習率の更新
        lr_scheduler.step()

        # もし開発用の評価セットが手元にある場合は、定期的に評価を実行しベストパフォーマンス時点のモデルを保持しておくと良い
        # https://github.com/hppRC/simple-simcse/blob/main/train.py#L301-L331

100%|██████████| 1562/1562 [16:35<00:00,  1.57it/s]


In [13]:
# 学習結果をDriveに保存しておく
torch.save(model, unsup_model_path)

# 3. 教師あり学習（sup-SimCSE）

NLI（自然言語推論）データセットの含意ペアを正例、矛盾ペアを負例として対照学習する。

注記: unsup-SimCSEと実装は大体一緒なので、異なる点のみコメントしてます。

## 3.1 モデルインスタンスの生成

In [14]:
model_name = 'bert-base-uncased'
model = SimCSEModel(model_name).to(device)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

## 3.2 データセットの準備

In [15]:
# sup-SimCSE訓練用のデータセットをダウンロード

# 論文で実際に使用されたNLI（自然言語推論）データセットが使用できる
!mkdir -p ./datasets/sup-simcse
!wget https://huggingface.co/datasets/princeton-nlp/datasets-for-simcse/resolve/main/nli_for_simcse.csv
!mv ./nli_for_simcse.csv ./datasets/sup-simcse/train.csv

--2023-11-01 09:19:09--  https://huggingface.co/datasets/princeton-nlp/datasets-for-simcse/resolve/main/nli_for_simcse.csv
Resolving huggingface.co (huggingface.co)... 3.163.189.114, 3.163.189.74, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/princeton-nlp/datasets-for-simcse/0747687ec3594fa449d2004fd3757a56c24bf5f7428976fb5b67176775a68d48?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27nli_for_simcse.csv%3B+filename%3D%22nli_for_simcse.csv%22%3B&response-content-type=text%2Fcsv&Expires=1699089549&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5OTA4OTU0OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9wcmluY2V0b24tbmxwL2RhdGFzZXRzLWZvci1zaW1jc2UvMDc0NzY4N2VjMzU5NGZhNDQ5ZDIwMDRmZDM3NTdhNTZjMjRiZjVmNzQyODk3NmZiNWI2NzE3Njc3NWE2OGQ0OD9yZXNwb25zZS1jb250ZW50L

In [16]:
# NLIデータセットは各エントリが以下の3つ組
#
#  - sent0: premise (前提文)
#  - sent1: entailment (含意)
#  - hard_neg: contradiction (矛盾)
#
# (premise, entailment)を正例、(premise, contradiction)を負例として学習する
#
# unsup-SimCSEと同様、ミニバッチ内の別の事例同士も負例として学習する
# （スライド25Pの右図が分かりやすい）

train_examples = pd.read_csv('./datasets/sup-simcse/train.csv')
train_examples

,sent0,sent1,hard_neg
0,you know during the season and i guess at at y...,You lose the things to the following level if ...,They never perform recalls on anything.
1,One of our number will carry out your instruct...,A member of my team will execute your orders w...,We have no one free at the moment so you have ...
2,How do you know? All this is their information...,This information belongs to them.,They have no information at all.
3,yeah i tell you what though if you go price so...,The tennis shoes can be in the hundred dollar ...,The tennis shoes are not over hundred dollars.
4,my walkman broke so i'm upset now i just have ...,I'm upset that my walkman broke and now I have...,My walkman still works as well as it always did.
...,...,...,...
275596,A group of four kids stand in front of a statu...,four kids standing,the kids are seated
275597,a kid doing tricks on a skateboard on a bridge,a kid is skateboarding,a kid is inside
275598,A dog with a blue collar plays ball outside.,a dog is outside,a dog is on the couch
275599,Four dirty and barefooted children.,four children have dirty feet.,four kids won awards for 'cleanest feet'


In [17]:
# sup-SimCSEでは3つ組を事例として返す
class SupSimCSEDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        premise: list[str],
        entailment: list[str],
        contradiction: list[str],
    ):
        assert len(premise) == len(entailment) == len(contradiction)
        self.premise = premise
        self.entailment = entailment
        self.contradiction = contradiction

    def __getitem__(self, index: int) -> tuple[str, str, str]:
        return self.premise[index], self.entailment[index], self.contradiction[index]

    def __len__(self) -> int:
        return len(self.premise)


# 学習に時間が掛かりすぎるので10万件に抑える
train_examples = train_examples[:100000]

train_dataset = SupSimCSEDataset(
    premise=train_examples['sent0'].tolist(),
    entailment=train_examples['sent1'].tolist(),
    contradiction=train_examples['hard_neg'].tolist(),
)

In [18]:
def tokenize(batch: list[str]) -> transformers.tokenization_utils.BatchEncoding:
    return tokenizer(
        batch,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=32,
    )


def collate_fn(batch: list[tuple[str, str, str]]) -> transformers.tokenization_utils.BatchEncoding:
    premise, entailment, contradiction = zip(*batch)
    return transformers.tokenization_utils.BatchEncoding(
        {
            'premise': tokenize(premise),
            'entailment': tokenize(entailment),
            'contradiction': tokenize(contradiction),
        }
    )

# 論文では、sup-SimCSEのミニバッチのサイズは512
# しかしここでは、メモリ使用量の関係で64
# https://github.com/princeton-nlp/SimCSE/tree/0.4#training
batch_size = 64

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    collate_fn=collate_fn,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    drop_last=True,
)

## 3.3 ファインチューニング

In [19]:
# 論文ではsup-SimCSEのエポック数は3だが、ここでは時間の都合上1 (付録A参照)
epochs = 1

# sup-SimCSEのbert-base-uncasedでの学習率は5e-5
# https://github.com/princeton-nlp/SimCSE/tree/0.4#training
learning_rate = 5e-5

temperature = 0.05

optimizer = torch.optim.AdamW(
    params=model.parameters(),
    lr=learning_rate
)

lr_scheduler = transformers.optimization.get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader) * epochs,
)

for epoch in range(epochs):
    model.train()

    for batch in tqdm.tqdm(train_dataloader):
        batch = batch.to(device)

        # それぞれの文について埋め込みを計算
        emb_pre = model.forward(**batch['premise'])
        emb_ent = model.forward(**batch['entailment'])
        emb_cnt = model.forward(**batch['contradiction'])

        # (emb_pre, emb_ent)と(emb_pre, emb_cnt)のそれぞれについて全対で類似度を計算し、最後に連結させる
        # スライドP25の右を作ってると思えば分かりやすい
        sim_matrix_pe = torch.nn.functional.cosine_similarity(emb_pre.unsqueeze(1), emb_ent.unsqueeze(0), dim=-1)
        sim_matrix_pc = torch.nn.functional.cosine_similarity(emb_pre.unsqueeze(1), emb_cnt.unsqueeze(0), dim=-1)
        sim_matrix = torch.cat([sim_matrix_pe, sim_matrix_pc], dim=1)

        sim_matrix = sim_matrix / temperature

        labels = torch.arange(batch_size).long().to(device)
        loss = torch.nn.functional.cross_entropy(sim_matrix, labels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        lr_scheduler.step()

100%|██████████| 1562/1562 [22:40<00:00,  1.15it/s]


In [20]:
# 学習結果をDriveに保存しておく
torch.save(model, sup_model_path)

# 4. 評価

STS (semantic textual similarity) Taskで埋め込みモデルの性能を評価する。

In [21]:
# 訓練済みSimCSEモデルを読み込む

unsup_model = None
if os.path.exists(unsup_model_path):
    unsup_model = torch.load(unsup_model_path)
else:
    print(f'{unsup_model_path} does not exist.')

sup_model = None
if os.path.exists(sup_model_path):
    sup_model = torch.load(sup_model_path)
else:
    print(f'{sup_model_path} does not exist.')

In [22]:
# ファインチューニングして無いモデルも比較用に作成

untuned_model = SimCSEModel('bert-base-uncased').to(device)

In [23]:
# STS Benchmarkデータセットを使用する。
# https://ixa2.si.ehu.eus/stswiki/index.php/STSbenchmark

!wget http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
!tar -zxvf Stsbenchmark.tar.gz
!mkdir -p ./datasets/sts
!mv stsbenchmark ./datasets/sts/stsb
!rm Stsbenchmark.tar.gz

--2023-11-01 09:42:00--  http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.es (ixa2.si.ehu.es)... 158.227.106.100
Connecting to ixa2.si.ehu.es (ixa2.si.ehu.es)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz [following]
--2023-11-01 09:42:01--  http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.eus (ixa2.si.ehu.eus)... 158.227.106.100
Connecting to ixa2.si.ehu.eus (ixa2.si.ehu.eus)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409630 (400K) [application/x-gzip]
Saving to: ‘Stsbenchmark.tar.gz’

Stsbenchmark.tar.gz 100%[===================>] 400.03K   394KB/s    in 1.0s    

2023-11-01 09:42:02 (394 KB/s) - ‘Stsbenchmark.tar.gz’ saved [409630/409630]

stsbenchmark/
stsbenchmark/readme.txt
stsbenchmark/sts-test.csv
stsbenchmark/correlation.pl
stsbenchmark/LICENSE.txt
stsb

In [24]:
# STS Benchmarkデータセットをパース
#
# 色々と列が含まれているが使用するのは sentence1, sentence2, score のみ
# scoreには、人手評価により決めたsentence1とsentence2の意味的な類似度がアノテーションされている

names = ['genre', 'file', 'year', 'sid', 'score', 'sentence1', 'sentence2']
sts_test_df = pd.read_csv(
    'datasets/sts/stsb/sts-test.csv',
    sep='\t',
    header=None,
    names=names,
    # オプショナルで追加列が存在するので、パースする列数を指定する必要あり
    usecols=range(len(names)),
    # エラー「ParserError: Error tokenizing data. C error: EOF inside string starting at row 1118.」に対処
    # https://stackoverflow.com/questions/18016037/pandas-parsererror-eof-character-when-reading-multiple-csv-files-to-hdf5
    quoting=csv.QUOTE_NONE,
)
sts_test_df

,genre,file,year,sid,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,24,2.5,A girl is styling her hair.,A girl is brushing her hair.
1,main-captions,MSRvid,2012test,33,3.6,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.
2,main-captions,MSRvid,2012test,45,5.0,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.
3,main-captions,MSRvid,2012test,63,4.2,A man is cutting up a cucumber.,A man is slicing a cucumber.
4,main-captions,MSRvid,2012test,66,1.5,A man is playing a harp.,A man is playing a keyboard.
...,...,...,...,...,...,...,...
1374,main-news,headlines,2016,1354,0.0,"Philippines, Canada pledge to further boost re...",Philippines saves 100 after ferry sinks
1375,main-news,headlines,2016,1360,1.0,Israel bars Palestinians from Jerusalem's Old ...,"Two-state solution between Palestinians, Israe..."
1376,main-news,headlines,2016,1368,1.0,How much do you know about Secret Service?,Lawmakers from both sides express outrage at S...
1377,main-news,headlines,2016,1420,0.0,Obama Struggles to Soothe Saudi Fears As Iran ...,Myanmar Struggles to Finalize Voter Lists for ...


In [25]:
# ミニバッチのサイズ
batch_size = 512

# 受け取ったSimCSEモデルを使って、入力文を埋め込みに変換する
#
# inference_modeを指定することで、評価時には余分な勾配の計算ための処理をスキップできる
# https://pytorch.org/docs/stable/generated/torch.inference_mode.html
@torch.inference_mode()
def encode(model: SimCSEModel, texts: list[str]) -> torch.Tensor:
    # 評価モードに切り替え
    # https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.eval
    model.eval()

    embs = []
    for batch in more_itertools.chunked(texts, batch_size):
        batch = tokenizer(
            batch,
            padding=True,
            truncation=True,
            return_tensors='pt',
        )
        batch = batch.to(device)
        emb = model(**batch)
        embs.append(emb.cpu())

    # shape of output: (len(texts), hidden_size)
    return torch.cat(embs, dim=0)

In [26]:
# モデルから得られた文埋め込み間のコサイン類似度と正解スコアを比較して、
# Spearmanの順位相関係数（×100）を性能スコアとして返す。
#
# Pearsonの相関係数も伝統的に使用されてきたが、Spearmanの方が文埋め込みの評価に適しているという議論がある
# （論文付録B参照）
def evaluate(model: SimCSEModel) -> float:
    sentences1 = sts_test_df['sentence1']
    sentences2 = sts_test_df['sentence2']
    scores = sts_test_df['score']

    embeddings1 = encode(model, sentences1)
    embeddings2 = encode(model, sentences2)

    cosine_scores = 1 - sklearn_metrics.pairwise.paired_cosine_distances(embeddings1, embeddings2)
    spearman = float(scipy.stats.spearmanr(scores, cosine_scores)[0]) * 100

    return spearman

In [27]:
print(f'notrain-simcse: {evaluate(untuned_model):g}')
if unsup_model is not None:
    print(f'unsup-simcse: {evaluate(unsup_model):g}')
if sup_model is not None:
    print(f'sup-simcse: {evaluate(sup_model):g}')

notrain-simcse: 20.2978
unsup-simcse: 61.5961
sup-simcse: 80.6809
